In [1]:
import pickle
import transformers
import torch
import pandas as pd

if torch.cuda.is_available():
    device = torch.device('cuda')
    print('GPU available')
else:
    device = torch.device('cpu')
    print('Uh oh, GPU unavailable')

/anaconda/envs/py312/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


GPU available


In [2]:
def assemble_data() -> pd.DataFrame:
    biden_first: list[tuple[any, str]] = pickle.load(open("biden_first_half.pickle", 'rb'))
    biden_last = pickle.load(open("biden_last_half.pickle", 'rb'))
    biden_first.extend(biden_last)
    return pd.DataFrame(biden_first, columns=["datetime", "tweet"])

biden_df = assemble_data()